In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append('../')

## Dataset

In [3]:
import datasets
conll = datasets.load_dataset("conll2003")
CONLL_NER_TAGS = conll['train'].features['ner_tags'].feature.names
print(CONLL_NER_TAGS)
conll["test"][2]

Found cached dataset conll2003 (/Users/azatsultanov/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)


  0%|          | 0/3 [00:00<?, ?it/s]

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


{'id': '2',
 'tokens': ['AL-AIN', ',', 'United', 'Arab', 'Emirates', '1996-12-06'],
 'pos_tags': [22, 6, 22, 22, 23, 11],
 'chunk_tags': [11, 0, 11, 12, 12, 12],
 'ner_tags': [5, 0, 5, 6, 6, 0]}

In [4]:
sorted(list(set([tag.split('-')[-1] for tag in CONLL_NER_TAGS]) - set(['O'])))

['LOC', 'MISC', 'ORG', 'PER']

In [5]:
def preprocessing_dataset(dataset):
    result = {'X': [], 'y_true': []}
    for sample in dataset:
        result['X'].append(' '.join(sample['tokens']))
        result['y_true'].append([CONLL_NER_TAGS[tag] for tag in sample['ner_tags']])
    return result

In [6]:
from itertools import islice

train_data = preprocessing_dataset(conll['train'])
oos_data = preprocessing_dataset(conll['test'])

In [7]:
import numpy as np
import torch
from tqdm import tqdm
from transformers import (pipeline, 
        AutoModelForTokenClassification, AutoTokenizer, 
        BertForTokenClassification, BertTokenizer)


class ModelWrapper():
    def __init__(self, model, tokenizer, classes):
        self.model = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy=None)
        self.classes_ = classes

    def _unite_entities(self, entities):
        if len(entities) <= 1:
            return entities

        united_result = []
        cur_entity = {key: entities[0][key] for key in ['entity', 'word', 'start', 'end']}
        for entity in entities[1:]:
            if entity['word'].startswith('##'):
                cur_entity['word'] += entity['word'].lstrip('#')
                cur_entity['end'] = entity['end']
            else:
                united_result.append(cur_entity)
                cur_entity = {key: entity[key] for key in ['entity', 'word', 'start', 'end']}
        united_result.append(cur_entity)
        return united_result

    def _convert_entities_to_bio(self, tokens, entities):
        bio_tags = []
        cur_entity_idx = 0
        for token in tokens:
            if (cur_entity_idx < len(entities))\
                    and (token == entities[cur_entity_idx]['word']):
                bio_tags.append(entities[cur_entity_idx]['entity'])
                cur_entity_idx += 1
            else:
                bio_tags.append('O')
        return bio_tags

    def _postprocessing(self, tokens, model_output):
        entities = self._unite_entities(model_output)
        bio_tags = self._convert_entities_to_bio(tokens, entities)
        return bio_tags

    def predict(self, X):
        with torch.no_grad():
            ner_entitites = self.model(X)
            tags = []
            for text, entities in tqdm(zip(X, ner_entitites)):
                tags.append(self._postprocessing(text.split(), entities))
            return tags


# Load pretrained model and tokenizer for English NER task (dslim/bert-base-NER)
model_name = "dslim/bert-base-NER"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained(model_name)
wrapped_model = ModelWrapper(model, tokenizer, ['LOC', 'MISC', 'ORG', 'PER'])

In [8]:
# train_data['y_pred'] = wrapped_model.predict(train_data['X'])
# oos_data['y_pred'] = wrapped_model.predict(oos_data['X'])

In [9]:
# import pickle

# with open('./train_data.pkl', 'wb') as f:
#     pickle.dump(train_data, f)
# with open('./oos_data.pkl', 'wb') as f:
#     pickle.dump(oos_data, f)

In [10]:
import pickle

with open('./train_data.pkl', 'rb') as f:
    train_data = pickle.load(f)

with open('./oos_data.pkl', 'rb') as f:
    oos_data = pickle.load(f)

In [16]:
from sbe_vallib.sampler.supervised_sampler import SupervisedSampler
from sbe_vallib.scorer.nlp_scorer import NerScorer
from sbe_vallib.utils.metrics import NER_IOB_METRICS


sampler = SupervisedSampler(train=train_data, oos=oos_data)


scorer = NerScorer(metrics=NER_IOB_METRICS)
scores = scorer.calc_metrics(model=wrapped_model, sampler=sampler, data_type='oos')
scores

{'precision_score': {'LOC': 0.9363057324840764,
  'MISC': 0.804920913884007,
  'ORG': 0.8944262295081967,
  'PER': 0.9319965126416739,
  'micro': 0.9043806982432733,
  'macro': 0.8919123471294885,
  'weighted': 0.9064257739880867},
 'f1_score': {'LOC': 0.8043775649794801,
  'MISC': 0.7206923682140047,
  'ORG': 0.8562460765850596,
  'PER': 0.7735166425470332,
  'micro': 0.8017742730409069,
  'macro': 0.7887081630813945,
  'weighted': 0.800394646791491},
 'recall_score': {'LOC': 0.7050359712230215,
  'MISC': 0.6524216524216524,
  'ORG': 0.8211920529801324,
  'PER': 0.6611008039579468,
  'micro': 0.7200779036827195,
  'macro': 0.7099376201456882,
  'weighted': 0.7200779036827195},
 'support': {'LOC': 1668,
  'MISC': 702,
  'ORG': 1661,
  'PER': 1617,
  'micro': 5648,
  'macro': 5648,
  'weighted': 5648}}

In [18]:
from sbe_vallib import Validation

ner_validation = Validation(model=wrapped_model,
                            scorer=scorer,
                            sampler=sampler,
                            pipeline='../../src/sbe_vallib/nlp/pipelines/Config_44.xlsx')
res = ner_validation.validate()

/Users/azatsultanov/Programming/vallib/repo/vallib/src/sbe_vallib/parser.py:64: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  full_pipeline_df[set(code_columns) | set(


	Train/oos samples are too huge for independence test, calculating on 0.4287 fraction
(7499, 85)


100%|██████████| 20/20 [00:02<00:00,  9.80it/s]


In [28]:
res.keys()

dict_keys(['train_test_independence_test', 'test_key_metric', 'test_ner_metric'])